In [1]:
import numpy as np
import pandas as pd

from time import process_time
import time

import datetime 
import arrow

import sys
import os
import re 


In [2]:
import requests
from bs4 import BeautifulSoup 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

In [3]:
path = 'C:/Users/s0970/python/701網路爬蟲/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(path) #開啟

<ipython-input-4-60bd7adf2b89>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟


In [319]:
url = 'https://www.booking.com/hotel/is/b59.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&ucfs=1&arphpl=1&dest_id=97&dest_type=country&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=127&sr_order=popularity&srpvid=87685bc4d8ff0154&srepoch=1646831010&from=searchresults#hotelTmpl'
driver.get(url)

### 旅館類型

In [310]:
html = BeautifulSoup(driver.page_source, 'html.parser')

In [311]:
category = html.select("h2.hp__hotel-name span")[0].text
category

'飯店'

### 旅館名稱

In [312]:
name = html.select_one("h2#hp_hotel_name")
[s.extract() for s in name.select_one("span")] ##移除某標籤
name = re.sub("\n","",name.text)
name

'B59 Hotel（B59酒店）'

In [297]:
driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()

<ipython-input-297-ad353ae6f5ca>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()


In [281]:
html = BeautifulSoup(driver.page_source, 'html.parser')

### 旅客國家

In [282]:
html.select("#review_list_page_container > ul > li")[1].select("span.bui-avatar-block__subtitle")[0].text

'\n\n臺灣'

In [284]:
country = html.select("#review_list_page_container > ul > li")[1].select("span.bui-avatar-block__subtitle")[0].text
pattern = re.compile(r'\w+')
country = pattern.findall(country)[0]
country

'臺灣'

### 旅客評論時間

In [285]:
reviews_time = html.select("#review_list_page_container > ul > li")[3].select("span.c-review-block__date")[1].text
pattern = re.compile(r'(\d+) 年 (\d+) 月 (\d+) 日')
pattern.search(reviews_time).group(0)

'2019 年 10 月 8 日'

In [286]:
reviews_time = arrow.get(int(pattern.search(reviews_time).group(1)),int(pattern.search(reviews_time).group(2)),int(pattern.search(reviews_time).group(3))).format("YYYY-MM-DD")
reviews_time

'2019-10-08'

### 旅客評論標題

In [287]:
title = html.select("#review_list_page_container > ul > li")[3].select("h3.c-review-block__title")[0].text
title = re.sub("\n","",title)
title

'非常好'

### 旅客評論

In [288]:
html.select("#review_list_page_container > ul > li")[0].select('span.c-review__body')[0].get("lang")

'zh'

In [272]:
html = BeautifulSoup(driver.page_source, 'html.parser')

In [192]:
#html.select("#review_list_page_container > ul > li")[3].select('span.c-review__body--original')
html.select("#review_list_page_container > ul > li")[0].select('span.c-review__body') #>>> 中文

[<span class="c-review__body" lang="zh">服务 房间都还不错. 运气很好 在那边晚上还看到了极光.</span>]

In [120]:
reviews =  html.select("#review_list_page_container > ul > li")[3].select('span.c-review__body--original')[0].text +  html.select("#review_list_page_container > ul > li")[3].select('span.c-review__body--original')[1].text
reviews

'Modern, location and very good breakfastRoom was next to reconstruction area'

### 旅客評分

In [198]:
score = html.select("#review_list_page_container > ul > li")[3].select('div.bui-review-score__badge')[0].text
score

In [103]:
df = pd.DataFrame([],columns=["name","category","from_country","reviews_time","title","reviews","score"])

In [357]:
df.append({"name":name,"category":category,"from_country":country,"reviews_time":reviews_time,"title":title,"reviews":reviews,"score":score},ignore_index=True)

,name,category,from_country,reviews_time,title,reviews,score
0,Grandi by Center Hotels,飯店,愛爾蘭,2022-03-06,"Nice hotel, very good base for all of the acti...","Hotel was fine, its a great base just 10-15 mi...",8.0


### 旅館對應的url和留言次數

In [370]:
driver = webdriver.Chrome(path) #開啟

url = "https://www.booking.com/searchresults.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&tmpl=searchresults&ac_click_type=b&ac_position=0&class_interval=1&dest_id=97&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&lang=zh-tw&no_rooms=1&offset=975&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&soz=1&src=index&src_elem=sb&srpvid=f96c6fe9d0dd0096&ss=Iceland&ss_all=0&ss_raw=iceland&ssb=empty&sshis=0&lang_click=other&cdl=en-us&lang_changed=1"
driver.get(url)

<ipython-input-370-43e6425a9f8f>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟


In [379]:
hrefs = []
n_reviews = []
data_ = {}
html = BeautifulSoup(driver.page_source, 'html.parser')


for i in html.select("div[data-testid='property-card']"):
    if i.select("div._f8ff3180e div._4abc4c3d5")!=[]:
        hrefs.append(i.select("h3._23bf57b84 a")[0].attrs["href"])


pattern = re.compile(r'(\d+)')
for i in html.select("div._f8ff3180e div._4abc4c3d5"):
    if len(pattern.findall(i.text))>1:
        n_reviews.append(int(pattern.findall(i.text)[0])*1000 + int(pattern.findall(i.text)[1]))
    else:
        n_reviews.append(int(pattern.findall(i.text)[0]))


data_.update(dict(zip(hrefs,n_reviews)))

In [380]:
data_


{'https://www.booking.com/hotel/is/nortia-luxury-apartments.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&ucfs=1&arphpl=1&dest_id=97&dest_type=country&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=976&sr_order=popularity&srpvid=993698a31a7201b3&srepoch=1646689352&from=searchresults#hotelTmpl': 69,
 'https://www.booking.com/hotel/is/welcome-apartment.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&ucfs=1&arphpl=1&dest_id=97&dest_type=country&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=977&sr_order=popularity&srpvid=993698a31a7201b3&srepoch=1646689352&from=searchresults#hotelTmpl': 369,
 '

In [381]:
hrefs = []
n_reviews = []
data_ = {}
for i in range(40):
    try:
        html = BeautifulSoup(driver.page_source, 'html.parser')


        for i in html.select("div[data-testid='property-card']"):
            if i.select("div._f8ff3180e div._4abc4c3d5")!=[]:
                hrefs.append(i.select("h3._23bf57b84 a")[0].attrs["href"])


        pattern = re.compile(r'(\d+)')
        for i in html.select("div._f8ff3180e div._4abc4c3d5"):
            if len(pattern.findall(i.text))>1:
                n_reviews.append(int(pattern.findall(i.text)[0])*1000 + int(pattern.findall(i.text)[1]))
            else:
                n_reviews.append(int(pattern.findall(i.text)[0]))


        data_.update(dict(zip(hrefs,n_reviews)))

        driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()
        time.sleep(3)
    except:
        break


<ipython-input-381-4ae0c91f44c2>:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()


# 爬蟲函數

In [318]:
def hotel_url(country_url):

    driver = webdriver.Chrome(path) #開啟
    driver.get(url)
    
    hrefs = []
    n_reviews = []
    hotel_data = {}
    for i in range(40):
        try:
            html = BeautifulSoup(driver.page_source, 'html.parser')

            for i in html.select("div[data-testid='property-card']"):
                if i.select("div._f8ff3180e div._4abc4c3d5")!=[]:
                    hrefs.append(i.select("h3._23bf57b84 a")[0].attrs["href"])


            pattern = re.compile(r'(\d+)')
            for i in html.select("div._f8ff3180e div._4abc4c3d5"):
                if len(pattern.findall(i.text))>1:
                    n_reviews.append(int(pattern.findall(i.text)[0])*1000 + int(pattern.findall(i.text)[1]))
                else:
                    n_reviews.append(int(pattern.findall(i.text)[0]))



            hotel_data.update(dict(zip(hrefs,n_reviews)))

            driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()
            time.sleep(3)
        except:
            break
    
    return  hotel_data
    

In [319]:
url = "https://www.booking.com/searchresults.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&tmpl=searchresults&ac_click_type=b&ac_position=0&class_interval=1&dest_id=97&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&lang=zh-tw&no_rooms=1&offset=0&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&soz=1&src=index&src_elem=sb&srpvid=f96c6fe9d0dd0096&ss=Iceland&ss_all=0&ss_raw=iceland&ssb=empty&sshis=0&lang_click=other;cdl=en-us;lang_changed=1"
urls = hotel_url(url)

<ipython-input-318-79887280f515>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟
<ipython-input-318-79887280f515>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()


In [347]:
urls

{'https://www.booking.com/hotel/is/grandi-by-center-hotels.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&ucfs=1&arphpl=1&dest_id=97&dest_type=country&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=e74992fcebb600ae&srepoch=1646686458&from=searchresults#hotelTmpl': 574,
 'https://www.booking.com/hotel/is/centerhotel-laugavegur.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&aid=304142&ucfs=1&arphpl=1&dest_id=97&dest_type=country&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=e74992fcebb600ae&srepoch=1646686458&from=searchresults#hotelTmpl': 589,
 

In [266]:
def one_page_data(self,url):
    
    driver = webdriver.Chrome(self.path) #開啟
    driver.get(url)

    html = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(2)
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,"hp_hotel_name")))

    #旅館類型
    category = html.select("h2.hp__hotel-name span")[0].text

    #旅館名稱
    name = html.select_one("h2#hp_hotel_name")
    [s.extract() for s in name.select_one("span")] ##移除某標籤
    name = re.sub("\n","",name.text)


    ### 點擊Ajax評論
    driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
    time.sleep(3)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,"bui-f-font-heading")))


    df = []
    while(True):
        try:
            k = 0 
            html = BeautifulSoup(driver.page_source, 'html.parser') #翻頁後 初始化 html 介面內容
            while(True):
                try:
                    #旅客國家
                    country = html.select("#review_list_page_container > ul > li")[k].select("span.bui-avatar-block__subtitle")[0].text
                    pattern = re.compile(r'\w+')
                    country = pattern.findall(country)[0]

                    #旅客評論時間
                    reviews_time = html.select("#review_list_page_container > ul > li")[k].select("span.c-review-block__date")[1].text
                    pattern = re.compile(r'(\d+) 年 (\d+) 月 (\d+) 日')
                    pattern.search(reviews_time).group(0)
                    reviews_time = arrow.get(int(pattern.search(reviews_time).group(1)),int(pattern.search(reviews_time).group(2)),int(pattern.search(reviews_time).group(3))).format("YYYY-MM-DD")

                    #旅客評論標題
                    title = html.select("#review_list_page_container > ul > li")[k].select("h3.c-review-block__title")[0].text
                    title = re.sub("\n","",title)
                    
                    #旅客評論
                    language = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].get("lang")
                    
                    if (language=="zh" or language=="zh-tw"): #評論為中文
                        try:
                            a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].text
                            b = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[1].text
                            reviews = a + " " +  b
                        except:
                            a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].text
                            reviews = a
                    else:
                        try:
                            a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[0].text
                            b = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[1].text
                            reviews = a + " " +  b
                        except:
                            a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[0].text
                            reviews = a


                    if reviews==[]:
                        break

                    #旅客評分
                    score = html.select("#review_list_page_container > ul > li")[k].select('div.bui-review-score__badge')[0].text
                    
                    
                    df.extend([name,category,country,reviews_time,title,reviews,score])

                    k+=1

                except:
                    break

            
            driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click() 
            time.sleep(2)
            WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME,"bui-f-font-heading")))
        
        except:
            break

    return pd.DataFrame(np.array(df).reshape(-1,7),columns=["name","category","from_country","reviews_time","title","reviews","score"])



In [11]:
url = 'https://www.booking.com/hotel/is/grandi-by-center-hotels.html?aid=304142;label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ;sid=e12b90519bab632c3ffdcb360924a5c1;dest_id=97;dest_type=country;dist=0;group_adults=2;group_children=0;hapos=1;hpos=1;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1646668500;srpvid=f96c6fe9d0dd0096;type=total;ucfs=1&#hotelTmpl'
data = one_page_data(url)

<ipython-input-10-0249503989ec>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path) #開啟
<ipython-input-10-0249503989ec>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()


In [207]:
data

,name,category,from_country,reviews_time,title,reviews,score
0,Grandi by Center Hotels,飯店,美國,2021-10-16,Exceptional visit!,This was a high service experience staying in ...,10
1,Grandi by Center Hotels,飯店,德國,2022-03-06,好極了,"modern, cooler Style, tolles Restaurant",9.0
2,Grandi by Center Hotels,飯店,英國,2022-03-06,Excellent hotel,"Great location, quiet, clean, spacious, comfy ...",10
3,Grandi by Center Hotels,飯店,愛爾蘭,2022-03-06,"Nice hotel, very good base for all of the acti...","Hotel was fine, its a great base just 10-15 mi...",8.0
4,Grandi by Center Hotels,飯店,冰島,2022-03-05,非常好,"Comfortable bed, friendly staff and great loca...",8.0
...,...,...,...,...,...,...,...
305,Grandi by Center Hotels,飯店,英國,2021-07-31,好,Fengum nýtt herbergi þegar fyrra herbergið var...,7.0
306,Grandi by Center Hotels,飯店,拉脫維亞,2021-09-07,傑出,"У отеля невероятно хорошее местоположение , ря...",10
307,Grandi by Center Hotels,飯店,奧地利,2021-08-28,Super Unterkunft in Islands Hauptstadt,Das Personal war ausgesprochen freundlich und ...,10
308,Grandi by Center Hotels,飯店,德國,2021-09-30,好,Das Bad war schönDas Frühstück,7.0


# 包成物件導向class

In [6]:
class booking_crawler:

    def __init__(self,path,country_url):
        self.path = path 
        self.country_url = country_url

    def hotel_url(self):
    
        driver = webdriver.Chrome(self.path) #開啟
        driver.get(self.country_url)
        
        hrefs = []
        n_reviews = []
        hotel_data = {}
        for i in range(40):
            try:
                html = BeautifulSoup(driver.page_source, 'html.parser')

                for i in html.select("div[data-testid='property-card']"):
                    if i.select("div._f8ff3180e div._4abc4c3d5")!=[]:
                        hrefs.append(i.select("h3._23bf57b84 a")[0].attrs["href"])


                pattern = re.compile(r'(\d+)')
                for i in html.select("div._f8ff3180e div._4abc4c3d5"):
                    if len(pattern.findall(i.text))>1:
                        n_reviews.append(int(pattern.findall(i.text)[0])*1000 + int(pattern.findall(i.text)[1]))
                    else:
                        n_reviews.append(int(pattern.findall(i.text)[0]))




                hotel_data.update(dict(zip(hrefs,n_reviews)))

                driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()
                time.sleep(3)
            except:
                break
        
        return  hotel_data

    def one_page_data(self,url):
    
        driver = webdriver.Chrome(self.path) #開啟
        driver.get(url)

        html = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(2)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,"hp_hotel_name")))

        #旅館類型
        category = html.select("h2.hp__hotel-name span")[0].text

        #旅館名稱
        name = html.select_one("h2#hp_hotel_name")
        [s.extract() for s in name.select_one("span")] ##移除某標籤
        name = re.sub("\n","",name.text)


        ### 點擊Ajax評論
        driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
        time.sleep(3)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,"bui-f-font-heading")))
    

        df = []
        while(True):
            try:
                k = 0 
                html = BeautifulSoup(driver.page_source, 'html.parser') #翻頁後 初始化 html 介面內容
                while(True):
                    try:
                        #旅客國家
                        country = html.select("#review_list_page_container > ul > li")[k].select("span.bui-avatar-block__subtitle")[0].text
                        pattern = re.compile(r'\w+')
                        country = pattern.findall(country)[0]

                        #旅客評論時間
                        reviews_time = html.select("#review_list_page_container > ul > li")[k].select("span.c-review-block__date")[1].text
                        pattern = re.compile(r'(\d+) 年 (\d+) 月 (\d+) 日')
                        pattern.search(reviews_time).group(0)
                        reviews_time = arrow.get(int(pattern.search(reviews_time).group(1)),int(pattern.search(reviews_time).group(2)),int(pattern.search(reviews_time).group(3))).format("YYYY-MM-DD")

                        #旅客評論標題
                        title = html.select("#review_list_page_container > ul > li")[k].select("h3.c-review-block__title")[0].text
                        title = re.sub("\n","",title)
                        
                        #旅客評論
                        language = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].get("lang")
                        
                        if (language=="zh" or language=="zh-tw"): #評論為中文
                            try:
                                a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].text
                                b = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[1].text
                                reviews = a + " " +  b
                            except:
                                a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body')[0].text
                                reviews = a
                        else:
                            try:
                                a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[0].text
                                b = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[1].text
                                reviews = a + " " +  b
                            except:
                                a = html.select("#review_list_page_container > ul > li")[k].select('span.c-review__body--original')[0].text
                                reviews = a


                        if (reviews==[] or reviews=="客人沒有留下任何評語。") :
                            break


                        #旅客評分
                        score = html.select("#review_list_page_container > ul > li")[k].select('div.bui-review-score__badge')[0].text

                        df.extend([name,category,country,reviews_time,title,reviews,score])
                                    
                        k+=1
                    except:
                        break

                
                driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click() 
                time.sleep(2)
                WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME,"bui-f-font-heading")))
            
            except:
                break

        return pd.DataFrame(np.array(df).reshape(-1,7),columns=["name","category","from_country","reviews_time","title","reviews","score"])

    

# 冰島

In [13]:
path = 'C:/Users/s0970/python/701網路爬蟲/chromedriver.exe'
url = "https://www.booking.com/searchresults.zh-tw.html?label=gen173bo-1DCAEoggI46AdIMFgDaOcBiAEBmAEwuAEXyAEM2AED6AEB-AEDiAIBmAICqAIDuALMrZiRBsACAdICJDA4MWM3ODVkLWU3YjYtNGUwNC1hOGMzLWJiMzJhYjZlODBhNdgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&tmpl=searchresults&ac_click_type=b&ac_position=0&class_interval=1&dest_id=97&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&lang=zh-tw&no_rooms=1&offset=0&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&soz=1&src=index&src_elem=sb&srpvid=f96c6fe9d0dd0096&ss=Iceland&ss_all=0&ss_raw=iceland&ssb=empty&sshis=0&lang_click=other;cdl=en-us;lang_changed=1"

crawler_ = booking_crawler(path,url)

In [5]:
n_url = crawler_.hotel_url()

<ipython-input-3-d28c702f67e0>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-3-d28c702f67e0>:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()


In [7]:
key_hotel = []
for url,n_click in n_url.items():
    if n_click>1000:
        key_hotel.append(url)

len(key_hotel)

90

In [ ]:
pd.DataFrame(key_hotel).to_csv("ice_url.csv",encoding="utf-8-sig")

### 旅館url

In [14]:
ice_url = pd.read_csv("ice_url.csv",encoding="utf-8",index_col=0)
ice_url = ice_url["0"]

In [15]:
ice_url

0     https://www.booking.com/hotel/is/oddsson.zh-tw...
1     https://www.booking.com/hotel/is/reykjavik-res...
2     https://www.booking.com/hotel/is/von.zh-tw.htm...
3     https://www.booking.com/hotel/is/siglo.zh-tw.h...
4     https://www.booking.com/hotel/is/bakki-hostel-...
                            ...                        
85    https://www.booking.com/hotel/is/laxa.zh-tw.ht...
86    https://www.booking.com/hotel/is/horgsland-gue...
87    https://www.booking.com/hotel/is/fosshotel-gla...
88    https://www.booking.com/hotel/is/skyrhusid-gue...
89    https://www.booking.com/hotel/is/horgsland-cot...
Name: 0, Length: 90, dtype: object

### 0~9

In [16]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[:10]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-6-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-6-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-6-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增


In [17]:
ice_land.to_csv("ice_land_10.csv",encoding="utf-8-sig")

In [43]:
pd.read_csv("ice_land_10.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Center Hotels Plaza                               1996
201 Hotel                                         1629
ODDSSON Hotel                                     1184
Reykjavik Residence Apartment Hotel（雷克雅未克公寓酒店）     925
Hotel Kjarnalundur（賈爾納倫杜酒店）                        896
Reykjavik Downtown Hotel                           800
Bakki Apartments & Hostel                          772
Sólheimahjáleiga Guesthouse（索爾海馬夏列卡農家樂）            760
Lighthouse-Inn（燈塔旅館）                               663
Siglo Hotel by Keahotels                           550
Name: name, dtype: int64

In [44]:
len(pd.read_csv("ice_land_10.csv",encoding="utf-8",index_col=0)["name"].unique())

10

### 10~19

In [20]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[10:20]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-6-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-6-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-6-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增


In [21]:
ice_land.to_csv("ice_land_20.csv",encoding="utf-8-sig")

In [86]:
pd.read_csv("ice_land_20.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Midgardur by Center Hotels          1550
Guesthouse Nypugardar（尼部卡搭賓館）       1169
Hotel Frón（芙蓉酒店）                    1160
Hotel Hafnarfjall（哈芬納爾福爾酒店）          975
Center Hotels Arnarhvoll             937
Eyja Guldsmeden Hotel（艾加古斯米登酒店）      796
Hotel South Coast                    787
Centrum Hotel                        703
Icelandair Hotel Herad（赫德冰島航空酒店）     569
Name: name, dtype: int64

In [87]:
len(pd.read_csv("ice_land_20.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(12)

9

### 20~29

In [9]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[20:30]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-4-8b4df1bb4acd>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-4-8b4df1bb4acd>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-4-8b4df1bb4acd>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增


In [42]:
ice_land.to_csv("ice_land_30.csv",encoding="utf-8-sig")

In [9]:
pd.read_csv("ice_land_30.csv",encoding="utf-8",index_col=0)["name"].value_counts()

BB Hótel by Reykjavik Keflavik Airport               2056
Icelandair Hotel Reykjavik Natura（雷克雅未克納圖拉冰島航空酒店）     975
Eric the Red Guesthouse（埃里克紅旅館）                       835
Galaxy Pod Hostel（銀河波德旅舍）                             824
Hotel Vík í Mýrdal                                    697
Sel - Hótel Mývatn（賽爾-米瓦特酒店）                          682
Icelandair Hotel Reykjavik Marina（冰島雷克雅未克濱海酒店）        663
Igdlo Guesthouse（伊格德羅旅館）                              586
Hotel Kea by Keahotels（凱亞凱亞酒店）                        566
B59 Hotel（B59酒店）                                      564
Name: name, dtype: int64

In [11]:
len(pd.read_csv("ice_land_30.csv",encoding="utf-8",index_col=0)["name"].unique())

10

### 30~49

In [43]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[30:50]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-12-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-12-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-12-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增
第10筆資料成功新增
第11筆資料成功新增
第12筆資料成功新增
第13筆資料成功新增
第14筆資料成功新增
第15筆資料成功新增
第16筆資料成功新增
第17筆資料成功新增
第18筆資料成功新增
第19筆資料成功新增


In [44]:
ice_land.to_csv("ice_land_50.csv",encoding="utf-8-sig")

In [12]:
pd.read_csv("ice_land_50.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Adventure Hotel Hof                         1526
Hotel Katla by Keahotels                    1135
Stay Apartments Bolholt（博爾霍爾特公寓酒店）          1114
Downtown Reykjavík Apartments（雷克雅未克中心公寓）    1099
Guesthouse Carina（卡瑞納旅館）                    1000
Hotel Ísland – Spa & Wellness Hotel          971
Lilja Guesthouse（麗杰旅館）                       922
Seljavellir Guesthouse（瑟爾杰維利酒店）              891
Stay Apartments Einholt（埃因霍特公寓酒店）            806
Hotel Jökull（冰川酒店）                           783
Kirkjufell Guesthouse and Apartments         763
Milk Factory（牛奶廠住宿加早餐旅館）                     749
KEF Guesthouse by Keflavík airport           717
22 Hill Hotel（22山酒店）                         698
Guesthouse Galtafell（加爾塔弗爾酒店）                688
Guesthouse Bitra B&B（比特拉住宿加早餐旅館）             646
Hotel Viking（維京酒店）                           623
Nordurey Hotel City Garden（北歐城市花園酒店）         621
Name: name, dtype: int64

In [13]:
len(pd.read_csv("ice_land_50.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(33,36)

18

### 50~59

In [27]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[50:60]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-3-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-3-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-3-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增


In [28]:
ice_land.to_csv("ice_land_60.csv",encoding="utf-8-sig")

In [29]:
pd.read_csv("ice_land_60.csv",encoding="utf-8",index_col=0)["name"].value_counts()

The Barn                                                                   2141
Hali Country Hotel（哈利鄉村酒店）                                                 1080
B&B Guesthouse - Bed and Breakfast Keflavik Centre（凱夫拉維克中心食宿酒店）             856
Kex Hostel（客可思旅舍）                                                           825
Skútustadir Guesthouse（斯庫圖斯塔德農家樂）                                           794
Guesthouse Aurora（極光賓館）                                                     767
Fossatun Camping Pods & Cottages - Sleeping Bag Accommodation（福薩頓睡袋露營地）     762
Hotel Berg by Keflavik Airport                                              666
Baldursbrá Guesthouse Laufásvegur（樂法維格寶德斯巴旅館）                               610
Akureyri Hostel                                                             547
Name: name, dtype: int64

In [30]:
len(pd.read_csv("ice_land_60.csv",encoding="utf-8",index_col=0)["name"].unique())

10

### 60~89

In [53]:
ice_land = pd.DataFrame([])

for i,url in enumerate(ice_url[60:90]):
    i+1
    try:
        ice_land = ice_land.append(crawler_.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-12-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-12-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-12-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增
第10筆資料成功新增
第11筆資料成功新增
第12筆資料成功新增
第13筆資料成功新增
第14筆資料成功新增
第15筆資料成功新增
第16筆資料成功新增
第17筆資料成功新增
第18筆資料成功新增
第19筆資料成功新增
第21筆資料成功新增
第22筆資料成功新增
第23筆資料成功新增
第24筆資料成功新增
第25筆資料成功新增
第27筆資料成功新增
第28筆資料成功新增
第29筆資料成功新增


In [54]:
ice_land.to_csv("ice_land_90.csv",encoding="utf-8-sig")

In [26]:
pd.read_csv("ice_land_90.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Aurora Hotel at Reykjavik-Keflavik Airport Terminal KEF    2412
Fosshotel Glacier Lagoon（冰河潟湖福斯酒店）                         1764
Hótel Kría（科里亞酒店）                                          1527
Black Beach Suites（黑色海灘套房公寓式酒店）                            1379
Hótel Laxá（拉夏酒店）                                           1120
Nupan Deluxe（豪華努潘住宿加早餐旅館）                                   995
Hörgsland Cottages（霍斯蘭德度假屋）                                 906
Klausturhof Guesthouse（克勞斯托霍夫旅館）                            877
Icelandair Hotel Myvatn（米湖冰島之空酒店）                           822
Skyrhúsid Guest House（斯吉琥西旅館）                               769
Skuggi Hotel by Keahotels                                   748
Hvoll Hostel                                                745
Selfoss Hostel（塞爾福斯旅館）                                      714
Lyngás Guesthouse（林佳斯賓館）                                    714
Puffin Hotel Vík（維克普費因酒店）                                   713
Hotel Höfn（赫本酒店）                        

In [25]:
len(pd.read_csv("ice_land_90.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(64,80,86)

27

# 希臘

In [7]:
path = 'C:/Users/s0970/python/701網路爬蟲/chromedriver.exe'
url2 ="https://www.booking.com/searchresults.zh-tw.html?label=gen173nr-1DCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AED6AEB-AECiAIBqAIDuAKggZqRBsACAdICJGI4NmI0ZjIwLTU3MWYtNGE0Ny05ZWQ2LTM0NWQ0MjEzZTY0NNgCBOACAQ&sid=e12b90519bab632c3ffdcb360924a5c1&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.zh-tw.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIMFgEaOcBiAEBmAEwuAEXyAEM2AED6AEB-AECiAIBqAIDuAKggZqRBsACAdICJGI4NmI0ZjIwLTU3MWYtNGE0Ny05ZWQ2LTM0NWQ0MjEzZTY0NNgCBOACAQ%3Bsid%3De12b90519bab632c3ffdcb360924a5c1%3Bsb_price_type%3Dtotal%26%3B&ss=Greece&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Greece&ac_position=0&ac_langcode=en&ac_click_type=b&dest_id=83&dest_type=country&place_id_lat=38.4727&place_id_lon=22.9792&search_pageview_id=6f5e9ad069c100f8&search_selected=true&search_pageview_id=6f5e9ad069c100f8&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0"

crawler_2 = booking_crawler(path,url2)

In [7]:
n2_url = crawler_2.hotel_url()

<ipython-input-5-7d9b770ca938>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-5-7d9b770ca938>:36: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="search_results_table"]/div/div/div/div/div[7]/div[2]/nav/div/div[3]/button/span').click()


In [8]:
key_hotel2 = []
for url,n_click in n2_url.items():
    if n_click>1000:
        key_hotel2.append(url)

len(key_hotel2)

74

In [96]:
pd.DataFrame(key_hotel2).to_csv("greece_url.csv",encoding="utf-8-sig")

In [8]:
greece_url = pd.read_csv("greece_url.csv",encoding="utf-8",index_col=0)
greece_url = greece_url["0"]

In [9]:
greece_url 

0     https://www.booking.com/hotel/gr/metropolis.zh...
1     https://www.booking.com/hotel/gr/athens-backpa...
2     https://www.booking.com/hotel/gr/lozenge.zh-tw...
3     https://www.booking.com/hotel/gr/the-athens-ga...
4     https://www.booking.com/hotel/gr/raise-acropol...
                            ...                        
70    https://www.booking.com/hotel/gr/hotellondonat...
71    https://www.booking.com/hotel/gr/coco-mat-athe...
72    https://www.booking.com/hotel/gr/athenscentreh...
73    https://www.booking.com/hotel/gr/faros1.zh-tw....
74    https://www.booking.com/hotel/gr/31-doors.zh-t...
Name: 0, Length: 75, dtype: object

### 0~19

In [34]:
greece = pd.DataFrame([])

for i,url in enumerate(greece_url[:20]):
    i+1
    try:
        greece = greece.append(crawler_2.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-3-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-3-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-3-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增
第10筆資料成功新增
第11筆資料成功新增
第12筆資料成功新增
第13筆資料成功新增
第14筆資料成功新增
第15筆資料成功新增
第16筆資料成功新增
第17筆資料成功新增
第18筆資料成功新增
第19筆資料成功新增


In [35]:
greece.to_csv("greece_20.csv",encoding="utf-8-sig")

In [19]:
pd.read_csv("greece_20.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Evripides Hotel（艾弗瑞派德酒店）          1948
Omiros Hotel（奧米洛斯酒店）              1799
Arethusa Hotel（阿瑞圖薩酒店）            1565
Plaka Hotel（普拉卡酒店）                1533
Athens Studios（雅典一室公寓）            1529
Athens Choice（艾森查爾斯旅館）            1428
Minoa Athens Hotel（雅典彌諾阿酒店）       1364
Hotel Lozenge（菱形酒店）               1171
Tempi Hotel（坦普酒店）                 1069
The Athens Gate Hotel（雅典門酒店）      1057
Piraeus Acropole Hotel            1014
Mandrino Hotel（曼德利諾酒店）             977
Mosaikon                           770
Athens Raise Acropolis Project     770
The Pinnacle Athens                756
Athens Mirabello（雅典米蘭貝洛酒店）         724
The Modernist Thessaloniki         636
Metropolis Hotel（大都市酒店）            630
Anita Hotel（安尼塔酒店）                 498
Name: name, dtype: int64

In [20]:
len(pd.read_csv("greece_20.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(7)

19

### 20~49

In [11]:
greece = pd.DataFrame([])

for i,url in enumerate(greece_url[20:50]):
    i+1
    try:
        greece = greece.append(crawler_2.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-6-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-6-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-6-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第7筆資料成功新增
第8筆資料成功新增
第9筆資料成功新增
第10筆資料成功新增
第11筆資料成功新增
第12筆資料成功新增
第13筆資料成功新增
第14筆資料成功新增
第15筆資料成功新增
第16筆資料成功新增
第17筆資料成功新增
第18筆資料成功新增
第21筆資料成功新增
第22筆資料成功新增
第23筆資料成功新增
第24筆資料成功新增
第25筆資料成功新增
第26筆資料成功新增
第27筆資料成功新增
第28筆資料成功新增
第29筆資料成功新增


In [12]:
greece.to_csv("greece_50.csv",encoding="utf-8-sig")

In [96]:
pd.read_csv("greece_50.csv",encoding="utf-8",index_col=0)["name"].value_counts()

Phidias Piraeus Hotel（菲迪亞斯比雷埃弗斯酒店）       2402
Infinity City Boutique Hotel（無限城精品酒店）    1877
Piraeus City Hotel                       1664
Hotel Orfeas（奧菲斯酒店）                      1630
Hotel El Greco（埃爾格雷柯酒店）                  1623
Amalia Hotel（阿瑪里亞酒店）                     1598
Amazon Hotel                             1505
Attalos Hotel（阿塔洛斯酒店）                    1449
Triton Hotel Piraeus（特里同比雷埃弗斯酒店）         1403
Blue Bottle Boutique Hotel（藍瓶精品酒店）       1389
Pella Inn Hostel（佩拉旅館）                   1348
Acropolis View Hotel（衛城景色酒店）             1342
Electra Hotel Athens（雅典伊萊克特拉酒店）          1310
El Greco Hotel（格列柯酒店）                    1070
Cecil Hotel（塞西爾酒店）                       1058
Adam's Hotel（亞當酒店）                        873
Agamemnon Hotel（阿伽門農酒店）                   868
Volos Palace（沃洛斯宮殿酒店）                     866
BED in Athens（雅典美宿酒店）                     851
Pyrgos Adrachti（皮爾格斯阿達奇迪酒店）               782
Lilia Hotel（莉利亞酒店）                        777
Crafoord Place Hostel Athens      

In [97]:
len(pd.read_csv("greece_50.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(39,40)

28

### 50~74

In [9]:
greece = pd.DataFrame([])
for i,url in enumerate(greece_url[50:75]):
    i+1
    try:
        greece = greece.append(crawler_2.one_page_data(url),ignore_index=True) 
        print("第{}筆資料成功新增".format(i))
    except:
        continue

<ipython-input-6-7d9b770ca938>:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path) #開啟
<ipython-input-6-7d9b770ca938>:62: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="guest-featured_reviews__horizontal-block"]/div/div[8]/button/span').click()
<ipython-input-6-7d9b770ca938>:124: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a').click()


第0筆資料成功新增
第1筆資料成功新增
第2筆資料成功新增
第3筆資料成功新增
第4筆資料成功新增
第5筆資料成功新增
第6筆資料成功新增
第8筆資料成功新增
第10筆資料成功新增
第11筆資料成功新增
第12筆資料成功新增
第13筆資料成功新增
第14筆資料成功新增
第15筆資料成功新增
第16筆資料成功新增
第17筆資料成功新增
第18筆資料成功新增
第19筆資料成功新增
第20筆資料成功新增
第21筆資料成功新增
第22筆資料成功新增
第23筆資料成功新增
第24筆資料成功新增


In [10]:
greece.to_csv("greece_75.csv",encoding="utf-8-sig")

In [11]:
pd.read_csv("greece_75.csv",encoding="utf-8",index_col=0)["name"].value_counts()

The Stanley（斯坦利酒店）                                                      3759
Plaza Hotel, Philian Hotels and Resorts（廣場酒店、菲律賓酒店及度假村）                 1857
Athinaiko Hotel（阿西恩尼克酒店）                                                1633
Infinity Blue Boutique Hotel & Spa - Adults Only（無限藍精品酒店及Spa - 僅限成人）    1328
Cosmopolit（凱帆酒店）                                                        1312
Athens Hawks                                                            1309
Titania Hotel（泰坦尼亞酒店）                                                   1188
Centrotel Hotel（薩特洛泰酒店）                                                 1068
Adrian Hotel（阿德里安酒店）                                                    1062
Athens21                                                                 862
Airotel Alexandros（愛若特亞里山德斯酒店）                                           816
Acropolis House（衛城酒店）                                                    775
City Hotel Thessaloniki（城市酒店）                                            749

In [12]:
len(pd.read_csv("greece_75.csv",encoding="utf-8",index_col=0)["name"].unique()) ##失敗(57,59)

23